In [ ]:
import numpy as np
import numpy.fft as fft

from scipy import signal as sg
from skimage import io

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import sys
sys.path.append("/home/tintin/rongheng/cv/lib/")

from image_lib import load_img, show_img, show_hist, resize_img
from signal_lib import bandpass
from edge_lib import threshold

img = load_img("bearing_scrached_fuji_tunnel.jpg")

In [ ]:
def bandpass_kernel(lowcut, highcut, transition, sample_frequency):
    bandpass_wavelet, length = bandpass(lowcut, highcut, transition, sample_frequency)
    bandpass_2d = np.outer(bandpass_wavelet, bandpass_wavelet)

    return bandpass_2d, length

In [ ]:
kernel, length = bandpass_kernel(500, 5, 1, sample_frequency=1000)
filtered_img = sg.convolve(kernel, img, 'full')
filtered_img = filtered_img[length:-length,length:-length]
filtered_img = threshold(filtered_img, -5.5, invert=True)

In [ ]:
show_img(filtered_img, filtered_img, size=(10,10), color="bwr")

In [ ]:
_, size = kernel.shape
X, Y = np.meshgrid(np.linspace(0,1,size), np.linspace(0,1,size))

fig = plt.figure()
ax = fig.gca(projection='3d')
ax.plot_surface(X, Y, kernel, rstride=1, cstride=1, cmap="gray", linewidth=0, aa=False)

In [ ]:
scratches = threshold(np.abs(filtered_img), .23, percent=True)
show_img(scratches,color=True,size=(12,12))

show_hist(scratches)

In [ ]:
fourier_raw = fft.fft2(filtered_img)
fourier_img = fft.fftshift(fourier_raw)
show_img((20*np.log10(0.1 + fourier_img)).astype(int), color=True) 

In [ ]:
# Morphology
binary_img = scratches.astype(bool)  # make binary
#binary_img = np.logical_not(binary_img)  # invert

#struct_elem = morph.generate_binary_structure(2, 1)
#print(struct_elem.astype(int))

#scratches = morph.binary_dilation(binary_img, struct_elem, 1)

labels, num_obj = measure.label(binary_img)
show_img(labels, color=True, size=(10,10))
show_hist(labels)

measure.find_objects(labels,max_label=3)

In [ ]:
# Points
resized_img = resize_img(img, scratches.shape)
points = get_points(scratches, min_dist=10, allow_cluster=True)
overlay_points(resized_img, points, size=(10, 10))

In [ ]:
# Count 
labels, num_scraches = measure.label(scratches_dial)
print("Number of Scraches: ", num_scraches)